In [ ]:
# import pandas library (can remove after integrating)
import pandas as pd

# use the average PWS of each country obtained from Question 1
sentiment_dict = {'AR': 56.28, 'CA': 75.57, 'CN': 63.05, 'JP': 80.77, 'US': 75.69} 

# use the total distance of each country obtained from Question 2
distance_dict = {'AR': 22495, 'CN': 5145492, 'JP': 1272783, 'CA': 5475049, 'US': 16583822}

# reuse the lst in Question 1
lst = ['AR', 'CA', 'CN', 'JP', 'US']

#### **Quicksort Algorithm**
###### To sort the final score obtained by each country
###### Running time complexity: O(n*log n)

In [ ]:
# Function to find the partition position
def partition(array, low, high):
  # Choose the rightmost element as pivot
  pivot = array[high]
  # Pointer for greater element
  i = low - 1
  # Traverse through all elements to compare each element with pivot
  for j in range(low, high):
    if array[j] <= pivot:
      # If element is smaller than pivot, swap it with the greater element pointed by i
      i = i + 1
      # Swapping element at i with element at j
      (array[i], array[j]) = (array[j], array[i])
  # Swap the pivot element with the greater element specified by i
  (array[i + 1], array[high]) = (array[high], array[i + 1])
  # Return the position from where partition is done
  return i + 1
 
# Function to perform quicksort
def quick_sort(array, low, high):
  if low < high:
    # Find pivot element (element < pivot are on the left, element > pivot are on the right)
    pi = partition(array, low, high)
    # Recursive call on the left of pivot
    quick_sort(array, low, pi - 1)
    # Recursive call on the right of pivot
    quick_sort(array, pi + 1, high)

#### **Perform min-max normalization**
##### Min-max normalization is one of the most common ways to normalize data. For every feature, the minimum value of that feature gets transformed into 0, the maximum value gets transformed into 1, and every other value gets transformed into a decimal between 0 and 1.

In [ ]:
# to get the minimum value in the dictionary
def getMin(dict):
  firstKey = str(list(dict.keys())[0])
  min = dict[firstKey]
  for key in dict:
    if (dict[key] < min):
      min = dict[key]
  return min

In [ ]:
# to get the maximum value in the dictionary
def getMax(dict):
  max = 0
  for key in dict:
    if (dict[key] > max):
      max = dict[key]
  return max

In [ ]:
def minMaxNormalization(min, max, value):
  return ((value - min) / (max - min))

#### **Apply weighted sum model**
##### Weighted sum model (WSM) is the best known and simplest multi-criteria decision analysis (MCDA) / multi-criteria decision making (MCDM) method for evaluating a number of alternatives in terms of a number of decision criteria.

In [ ]:
# Step 1: apply weight to each normalized score in the dictionary
def applyWeight(dict, weight):
  weighted_dict = {}
  for key in dict:
    weightedScore = dict[key] * weight
    weighted_dict[key] = weightedScore
  return weighted_dict

In [ ]:
# Step 2: sum up the weighted sentiment score and weighted distance score to obtain the final score for each country
def sumWeightedScore(weightedPWS_dict, weightedDistance_dict):
  finalScore_dict = {}
  for country in weightedPWS_dict:
    finalScore = round((weightedPWS_dict[country] + weightedDistance_dict[country]), 2)
    finalScore_dict[country] = finalScore
  return finalScore_dict

#### **Analyse ranking** 

In [ ]:
# calculate the probability of a country to be chosen to have an expansion among the 6 countries
def computeProbability(finalScore_list):
  totalFinalScore = sum(finalScore_list)
  probability_list = []
  for score in finalScore_list:
    probability = round((score / totalFinalScore), 4)
    probability_list.append(probability)
  return (dict(zip(lst, probability_list)))  

In [ ]:
# sort the countries based on their respective final score
def analyseRanking(finalScore_list, finalScore_dict):
  quick_sort(finalScore_list, 0, (len(finalScore_list)-1))
  rank_dict = {}
  for i in range(len(finalScore_list)-1, -1, -1):
    for country in finalScore_dict:
      if (finalScore_list[i] == finalScore_dict[country]):
        rank_dict[country] = finalScore_list[i]
  return rank_dict

In [ ]:
# display the ranking of countries where new stores can be located in the form of dataframe
def displayRanking(rank_dict, sentiment_dict, distance_dict, probability_dict):
  rank_num = [1, 2, 3, 4, 5]
  count = 0
  df = pd.DataFrame(columns = ['Ranking', 'Country', 'Avg PWS (%)', 'Total journey for deliveries (miles)', 'Score', 'Probability'])
  for country in rank_dict:
    df = df.append({'Ranking': rank_num[count],
                    'Country': country,
                    'Avg PWS (%)': sentiment_dict[country],
                    'Total journey for deliveries (miles)': distance_dict[country],
                    'Score': rank_dict[country],
                    'Probability': probability_dict[country]},
                    ignore_index = True)
    count += 1
  df = df.set_index("Ranking")
  print("Ranking of most recommended country to have expansion:\n")
  display(df)
  country_name = {'AR': 'Argentina', 'CA': 'Canada', 'CN': 'China', 'JP': 'Japan', 'US': 'United States'}
  print(f"\nThe most recommended country to have expansion is {country_name[list(rank_dict.keys())[0]]}.")
  print(f"The least recommended country to have expansion is {country_name[list(rank_dict.keys())[4]]}.")

#### **Driver code**

In [ ]:
# Perform min-max normalization on the average PWS obtained in Question 1
minPWS = getMin(sentiment_dict)
maxPWS = getMax(sentiment_dict)
normalizedPWS_dict = {}
for country in sentiment_dict:
  normalizedPWS = minMaxNormalization(minPWS, maxPWS, sentiment_dict[country])
  normalizedPWS_dict[country] = normalizedPWS

# Perform min-max normalization on the total distance obtained in Question 2
minDistance = getMin(distance_dict)
maxDistance = getMax(distance_dict)
normalizedDistance_dict = {}
for country in distance_dict:
  normalizedDistance = minMaxNormalization(maxDistance, minDistance, distance_dict[country])
  normalizedDistance_dict[country] = normalizedDistance

# Apply weighted sum model
# both weight of the sentiment and distance are the same (0.5)
weight_sentiment = 0.5
weight_distance = 0.5
weightedPWS_dict = applyWeight(normalizedPWS_dict, weight_sentiment)
weightedDistance_dict = applyWeight(normalizedDistance_dict, weight_distance)
finalScore_dict = sumWeightedScore(weightedPWS_dict, weightedDistance_dict)

# Compute probabiltiy of a country to be chosen to have expansion among the 6 countries
finalScore_list = list(finalScore_dict.values())
probability_dict = computeProbability(finalScore_list)

# Analyse ranking of countries using Quicksort Algorithm
rank_dict = analyseRanking(finalScore_list, finalScore_dict)

# Display the ranking
displayRanking(rank_dict, sentiment_dict, distance_dict, probability_dict)

Ranking of most recommended country to have expansion:



,Country,Avg PWS (%),Total journey for deliveries (miles),Score,Probability
Ranking,,,,,
1,JP,80.77,1272783,0.96,0.3127
2,CA,75.57,5475049,0.73,0.2378
3,AR,56.28,22495,0.50,0.1629
4,CN,63.05,5145492,0.48,0.1564
5,US,75.69,16583822,0.40,0.1303



The most recommended country to have expansion is Japan.
The least recommended country to have expansion is United States.


In [ ]:
# This cell is just for checking and tracing purpose. (can be deleted afterwards)

minPWS = getMin(sentiment_dict)
maxPWS = getMax(sentiment_dict)
#
print("min PWS:", minPWS)
print("max PWS:", maxPWS)
#
normalizedPWS_dict = {}
for country in sentiment_dict:
  normalizedPWS = minMaxNormalization(minPWS, maxPWS, sentiment_dict[country])
  normalizedPWS_dict[country] = normalizedPWS
#
print("normalized PWS dict:", normalizedPWS_dict)
#

minDistance = getMin(distance_dict)
maxDistance = getMax(distance_dict)
#
print("min dist:", minDistance)
print("max dist:", maxDistance)
#
normalizedDistance_dict = {}
for country in distance_dict:
  normalizedDistance = minMaxNormalization(maxDistance, minDistance, distance_dict[country])
  normalizedDistance_dict[country] = normalizedDistance
#
print("normalized dist dict:", normalizedDistance_dict)
#

weight_sentiment = 0.5
weight_distance = 0.5
weightedPWS_dict = applyWeight(normalizedPWS_dict, weight_sentiment)
weightedDistance_dict = applyWeight(normalizedDistance_dict, weight_distance)
#
print("weighted PWS dict:", weightedPWS_dict)
print("weighted dist dict:", weightedDistance_dict)
#
finalScore_dict = sumWeightedScore(weightedPWS_dict, weightedDistance_dict)
#
print("final score dict:", finalScore_dict)
#

finalScore_list = list(finalScore_dict.values())
probability_dict = computeProbability(finalScore_list)
#
print("probability dict:", probability_dict)
#
rank_dict = analyseRanking(finalScore_list, finalScore_dict)
print("rank dict:", rank_dict)

min PWS: 56.28
max PWS: 80.77
normalized PWS dict: {'AR': 0.0, 'CA': 0.7876684360963657, 'CN': 0.2764393630053082, 'JP': 1.0, 'US': 0.7925683952633729}
min dist: 22495
max dist: 16583822
normalized dist dict: {'AR': 1.0, 'CN': 0.6906650656677451, 'JP': 0.9245055664923469, 'CA': 0.6707658752224384, 'US': -0.0}
weighted PWS dict: {'AR': 0.0, 'CA': 0.39383421804818286, 'CN': 0.1382196815026541, 'JP': 0.5, 'US': 0.39628419763168643}
weighted dist dict: {'AR': 0.5, 'CN': 0.34533253283387255, 'JP': 0.46225278324617347, 'CA': 0.3353829376112192, 'US': -0.0}
final score dict: {'AR': 0.5, 'CA': 0.73, 'CN': 0.48, 'JP': 0.96, 'US': 0.4}
probability dict: {'AR': 0.1629, 'CA': 0.2378, 'CN': 0.1564, 'JP': 0.3127, 'US': 0.1303}
rank dict: {'JP': 0.96, 'CA': 0.73, 'AR': 0.5, 'CN': 0.48, 'US': 0.4}
